#### Imports

In [51]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

In [52]:
df = pd.read_csv('../../Datasets/final_dataset_descr.csv', sep='\t')

In [53]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0


#### Paso 1: Filtrar y eliminar productos con poca historia


Completamos el dataset con 0 para los producto / cliente que no existen

In [ ]:
# product_ids = [20001, 20002, 20003, 20004, 20005, 20006, 20007, 20008, 20009, 20010, 20011, 20012]

In [ ]:
# filtered_df = df[df['product_id'].isin(product_ids)]

In [ ]:
# display(filtered_df)

In [ ]:
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

df = df[df['periodo'] >= '2018-12-01']

product_info = df[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']].drop_duplicates()

min_max_periods = df.groupby(['customer_id', 'product_id'])['periodo'].agg(['min', 'max']).reset_index()

all_dfs = []

cont = 1

for _, row in min_max_periods.iterrows():
    customer_id = row['customer_id']
    product_id = row['product_id']
    min_period = row['min']
    max_period = '2019-12-01'
    all_periods = pd.date_range(min_period, max_period, freq='MS')
    
    combinations = pd.DataFrame({
        'customer_id': [customer_id] * len(all_periods),
        'product_id': [product_id] * len(all_periods),
        'periodo': all_periods
    })
    
    merged_df = pd.merge(combinations, df, on=['customer_id', 'product_id', 'periodo'], how='left')
    
    merged_df['tn'] = merged_df['tn'].fillna(0)
    
    merged_df['tn'] = merged_df['tn'].fillna(0)
    merged_df['cat1'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat1'])
    merged_df['cat2'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat2'])
    merged_df['cat3'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat3'])
    merged_df['brand'] = merged_df['product_id'].map(product_info.set_index('product_id')['brand'])
    merged_df['sku_size'] = merged_df['product_id'].map(product_info.set_index('product_id')['sku_size'])
    merged_df['descripcion'] = merged_df['product_id'].map(product_info.set_index('product_id')['descripcion'])
    
    merged_df['quarter'] = 'Q' + merged_df['periodo'].dt.to_period('Q').astype(str).str[-1]
    merged_df['month'] = merged_df['periodo'].dt.month.astype(str).str.zfill(2)
    
    merged_df['plan_precios_cuidados'] = merged_df['plan_precios_cuidados'].fillna(0)
    merged_df['cust_request_qty'] = merged_df['cust_request_qty'].fillna(0)
    merged_df['cust_request_tn'] = merged_df['cust_request_tn'].fillna(0)
    merged_df['close_quarter'] = merged_df['close_quarter'].fillna(0)
    merged_df['age'] = merged_df['age'].fillna(0)
    merged_df['mes_inicial'] = min_period
    
    all_dfs.append(merged_df)
    
    print(f"procesado {cont} de {len(min_max_periods)}")
    cont += 1

df_full = pd.concat(all_dfs, ignore_index=True)

df_full = df_full.sort_values(by=['customer_id', 'product_id', 'periodo'])

df_full['periodo'] = df_full['periodo'].dt.strftime('%Y%m')

display(df_full)

In [23]:
df_full.to_csv('final_dataset_completo_con_ceros.csv', sep='\t')

In [55]:
df_full = pd.read_csv("final_dataset_completo_con_ceros.csv", sep='\t')

In [56]:
display(df_full)

,Unnamed: 0,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,0,10001,20001,201812,0.0,20.0,254.62373,254.62373,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,12,1.0,23.0,2018-12-01
1,1,10001,20001,201901,0.0,53.0,393.26092,386.60688,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0.0,24.0,2018-12-01
2,2,10001,20001,201902,0.0,39.0,309.90610,309.90610,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,2,0.0,25.0,2018-12-01
3,3,10001,20001,201903,0.0,23.0,142.87158,130.54927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1.0,26.0,2018-12-01
4,4,10001,20001,201904,0.0,33.0,364.37071,364.37071,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,HC,PROFESIONAL,PISOS,MUSCULO,5000,Profesional menta,Q4,12,0.0,0.0,2019-11-01
2040579,2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,HC,PROFESIONAL,Gel,MUSCULO,5000,Industrial 5L,Q4,11,0.0,4.0,2019-11-01
2040580,2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,HC,PROFESIONAL,Gel,MUSCULO,5000,Industrial 5L,Q4,12,0.0,0.0,2019-11-01
2040581,2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,HC,PROFESIONAL,PISOS,MUSCULO,5000,Profesinal pisos plastificados,Q4,11,0.0,4.0,2019-11-01


In [57]:
df_full = df_full.drop(columns=['Unnamed: 0'])

#### Paso 2: Aplicar LabelEncoder a las columnas categóricas


In [58]:
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'quarter']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_full[col] = le.fit_transform(df_full[col])
    label_encoders[col] = le

display(df_full)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,1,8,54,21,5000,260,3,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,1,8,31,21,5000,158,3,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,1,8,31,21,5000,158,3,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,1,8,54,21,5000,256,3,11,0.0,4.0,2019-11-01


#### Paso 3: Agrupar y calcular el ratio de ventas por product_id


*NO SE USA EN ESTA VERSION, CODIGO HEREDADO DE CUANDO SE TRABAJO HASTA PRODUCTO SOLAMENTE*

In [61]:
grouped_sales = df_full.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'product_id'])['tn'].sum().reset_index()
grouped_sales = grouped_sales[grouped_sales['periodo'] == 201912]

group_totals = grouped_sales.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand'])['tn'].sum().reset_index()

ratios = pd.merge(grouped_sales, group_totals, on=['periodo', 'cat1', 'cat2', 'cat3', 'brand'], suffixes=('', '_total'))

ratios['ratio'] = ratios['tn'] / ratios['tn_total']

ratio_dict = ratios.set_index(['cat1', 'cat2', 'cat3', 'brand', 'product_id'])['ratio'].to_dict()


In [62]:
display(ratio_dict)

{(0, 0, 4, 22, 20609): 1.0,
 (0, 0, 11, 22, 20266): 0.8902853590993973,
 (0, 0, 11, 22, 20325): 0.09463369678765542,
 (0, 0, 11, 22, 20503): 0.015080944112947194,
 (0, 0, 41, 18, 20299): 1.0,
 (0, 0, 41, 19, 20158): 1.0,
 (0, 0, 41, 22, 20033): 0.5843286913353127,
 (0, 0, 41, 22, 20095): 0.17227855901276462,
 (0, 0, 41, 22, 20119): 0.21901984451738263,
 (0, 0, 41, 22, 20242): 0.02437290513454,
 (0, 0, 48, 18, 20107): 0.5835676012609781,
 (0, 0, 48, 18, 20161): 0.2368165294241783,
 (0, 0, 48, 18, 20175): 0.13365037261349158,
 (0, 0, 48, 18, 20270): 0.04596549670135207,
 (0, 0, 48, 19, 20084): 0.28608061827647613,
 (0, 0, 48, 19, 20116): 0.35835342288529287,
 (0, 0, 48, 19, 20121): 0.23859655876074223,
 (0, 0, 48, 19, 20227): 0.11696940007748882,
 (0, 0, 48, 22, 20003): 0.3082454843319254,
 (0, 0, 48, 22, 20004): 0.2203131836596504,
 (0, 0, 48, 22, 20005): 0.2048903667719819,
 (0, 0, 48, 22, 20019): 0.12141472640344769,
 (0, 0, 48, 22, 20046): 0.051790796091872884,
 (0, 0, 48, 22, 20108)

#### Paso 4: Agrupar ventas por periodo, cat1, cat2, cat3, brand, customer_id y product_id


Aplico escalado

In [65]:
scalers = {}
scaled_df = df_full.copy()

for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

joblib.dump(scalers, 'scalers.pkl')


['scalers.pkl']

In [67]:
df_full

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,1,8,54,21,5000,260,3,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,1,8,31,21,5000,158,3,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,1,8,31,21,5000,158,3,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,1,8,54,21,5000,256,3,11,0.0,4.0,2019-11-01


In [66]:
display(scaled_df)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,7.566192,112.767975,117.729042,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,20.632564,174.214121,178.798380,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,15.089255,137.269976,143.308502,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,8.754044,63.237695,60.319099,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,12.713551,161.409533,168.509580,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,-0.352821,-0.085197,-0.086775,1,8,54,21,5000,260,3,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,0.043130,-0.076847,-0.078058,1,8,31,21,5000,158,3,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,-0.352821,-0.085197,-0.086775,1,8,31,21,5000,158,3,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,0.043130,-0.077144,-0.078368,1,8,54,21,5000,256,3,11,0.0,4.0,2019-11-01


In [69]:
scaled_df.dtypes

customer_id                int64
product_id                 int64
periodo                    int64
plan_precios_cuidados    float64
cust_request_qty         float64
cust_request_tn          float64
tn                       float64
cat1                       int64
cat2                       int64
cat3                       int64
brand                      int64
sku_size                   int64
descripcion                int64
quarter                    int64
month                      int64
close_quarter            float64
age                      float64
mes_inicial               object
dtype: object

Agrupo y sumarizo

In [77]:
grouped_df = scaled_df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id', 'quarter', 'month']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()


In [78]:
display(grouped_df[(grouped_df["customer_id"] == 10001) & (grouped_df["product_id"] == 20001)])

,periodo,cat1,cat2,cat3,brand,customer_id,product_id,quarter,month,cust_request_qty,cust_request_tn,tn
15840,201812,1,10,47,0,10001,20001,3,12,7.566192,112.767975,117.729042
80495,201901,1,10,47,0,10001,20001,0,1,20.632564,174.214121,178.798380
175202,201902,1,10,47,0,10001,20001,0,2,15.089255,137.269976,143.308502
294172,201903,1,10,47,0,10001,20001,0,3,8.754044,63.237695,60.319099
431929,201904,1,10,47,0,10001,20001,1,4,12.713551,161.409533,168.509580
583070,201905,1,10,47,0,10001,20001,1,5,11.921650,194.888150,203.460397
744214,201906,1,10,47,0,10001,20001,1,6,2.418834,29.133498,30.416793
916759,201907,1,10,47,0,10001,20001,2,7,5.190488,64.086701,66.907040
1099078,201908,1,10,47,0,10001,20001,2,8,3.210735,14.824531,15.478599
1288478,201909,1,10,47,0,10001,20001,2,9,6.774291,49.340821,50.372396


Aplico DTW para agrupar los registros (series de categorias/clientes similares)

In [147]:
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt

pivoted_df = grouped_df.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'], columns='periodo', values='tn').fillna(0)

inertia = []
max_clusters = 12

for k in range(4, max_clusters + 1):
    print(f"Running K: {k}")
    model = TimeSeriesKMeans(n_clusters=k, metric="dtw", random_state=0)
    model.fit(pivoted_df.values)
    inertia.append(model.inertia_)


Running K: 4


KeyboardInterrupt: 

In [ ]:
plt.plot(range(4, max_clusters + 1), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

In [88]:
# Debug de los valores
max_value = pivoted_df[201812].max()
min_value = pivoted_df[201812].min()

print(f"Máximo valor de la columna 'nombre_de_la_columna': {max_value}")
print(f"Mínimo valor de la columna 'nombre_de_la_columna': {min_value}")


Máximo valor de la columna 'nombre_de_la_columna': 132.775343206
Mínimo valor de la columna 'nombre_de_la_columna': -0.08672881487933946


In [89]:
n_clusters = 15
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
cluster_labels = model.fit_predict(pivoted_df.values)

pivoted_df['cluster'] = cluster_labels

grouped_df = grouped_df.merge(pivoted_df['cluster'], left_on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'], right_index=True)

grouped_df.to_csv('grouped_with_clusters_normalized.csv', index=False)

display(grouped_df)

,periodo,cat1,cat2,cat3,brand,customer_id,product_id,quarter,month,cust_request_qty,cust_request_tn,tn,cluster
0,201812,0,0,4,22,10001,20609,3,12,2.022883,0.302771,0.318254,0
1,201812,0,0,4,22,10002,20609,3,12,2.814785,0.037928,0.041765,0
2,201812,0,0,4,22,10003,20609,3,12,0.043130,0.035605,0.039340,0
3,201812,0,0,4,22,10004,20609,3,12,0.043130,-0.024796,-0.023718,0
4,201812,0,0,4,22,10005,20609,3,12,2.418834,-0.057319,-0.057671,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,201912,3,13,82,32,10367,21222,3,12,-0.352821,-0.085197,-0.086775,0
2040579,201912,3,13,82,32,10482,21192,3,12,-0.352821,-0.085197,-0.086775,0
2040580,201912,3,13,82,32,10482,21222,3,12,-0.352821,-0.085197,-0.086775,0
2040581,201912,3,13,82,32,10513,21222,3,12,-0.352821,-0.085197,-0.086775,0


#### Paso 5: Armar un modelo LSTM


In [143]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.regularizers import l2

le_factor = 0.01

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(le_factor), input_shape=input_shape))
    model.add(Dropout(0.1))
    
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    
    
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    # model.add(Dropout(0.1))
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(128, activation='tanh', kernel_regularizer=l2(le_factor)))
    model.add(Dropout(0.1))

    model.add(Dense(128, activation='tanh', kernel_regularizer=l2(le_factor)))
    model.add(Dense(64, activation='tanh', kernel_regularizer=l2(le_factor))) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

#### Paso 6: Entrenar y predecir con el modelo LSTM para cada grupo


In [150]:
import pandas as pd
total = 0

for i in range(0, 15):
    cluster_number = i

    # Filtrar el DataFrame por el cluster deseado
    cluster_data = grouped_df[grouped_df['cluster'] == cluster_number]

    unique_combinations = cluster_data[['customer_id', 'product_id']].drop_duplicates().shape[0]
    total += unique_combinations
    print(f"Cluster {cluster_number}:")
    print(f"  Número de registros: {len(cluster_data)}")
    print(f"  Número de combinaciones únicas 'customer_id' y 'product_id': {unique_combinations}")
    print()
    
print(f"Total {total}")

Cluster 0:
  Número de registros: 1926903
  Número de combinaciones únicas 'customer_id' y 'product_id': 197234

Cluster 1:
  Número de registros: 2291
  Número de combinaciones únicas 'customer_id' y 'product_id': 180

Cluster 2:
  Número de registros: 13
  Número de combinaciones únicas 'customer_id' y 'product_id': 1

Cluster 3:
  Número de registros: 336
  Número de combinaciones únicas 'customer_id' y 'product_id': 28

Cluster 4:
  Número de registros: 65
  Número de combinaciones únicas 'customer_id' y 'product_id': 5

Cluster 5:
  Número de registros: 19812
  Número de combinaciones únicas 'customer_id' y 'product_id': 1644

Cluster 6:
  Número de registros: 13
  Número de combinaciones únicas 'customer_id' y 'product_id': 1

Cluster 7:
  Número de registros: 78
  Número de combinaciones únicas 'customer_id' y 'product_id': 6

Cluster 8:
  Número de registros: 364
  Número de combinaciones únicas 'customer_id' y 'product_id': 28

Cluster 9:
  Número de registros: 13
  Número de 

In [151]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np

models = {}

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001, verbose=1)
scaler_tn = scalers['tn']

# Preparar los datos por cluster
for cluster in range(n_clusters):
    display(f'Entrenando cluster numero: {cluster}')
    cluster_data = grouped_df[grouped_df['cluster'] == cluster].copy()
    cluster_data.sort_values(by='periodo', inplace=True)
    
    X, y = [], []
    for key, data in cluster_data.groupby(['customer_id', 'product_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'quarter', 'month', 'customer_id', 'product_id', 'tn']].values
        if len(series) > 2:  # Asegurarse de que haya suficientes datos
            X.append(series[:-2])  # Todos los datos excepto los últimos 2
            y.append(series[-1, -1])
            

    #Padleft para que todos los registros tengan el mismo shape
    max_len = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_len - len(seq), 0), (0, 0)), mode='constant') for seq in X]).astype(np.float32)
    y = np.array(y).astype(np.float32)
    
    # print(len((X_padded)))
    # print(len((y)))
    
    if len(X_padded) == 0 or len(y) == 0:
        continue
    
    validation_split = 0 if cluster in [2, 3, 4, 6, 7, 8, 9, 12, 13] else 0.2 #sino va a romper

    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

    # Construir y entrenar el modelo
    model = build_lstm_model((X_padded.shape[1], X_padded.shape[2]))
    model.fit(X_padded, y, epochs=500, verbose=2, batch_size=1000, validation_split=validation_split, callbacks=[reduce_lr, early_stopping])
    models[cluster] = model


'Entrenando cluster numero: 0'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
154/154 - 203s - 1s/step - loss: 2.7669 - val_loss: 0.0650 - learning_rate: 0.0010
Epoch 2/500
154/154 - 197s - 1s/step - loss: 0.0178 - val_loss: 9.2997e-04 - learning_rate: 0.0010
Epoch 3/500
154/154 - 196s - 1s/step - loss: 0.0030 - val_loss: 3.5409e-04 - learning_rate: 0.0010
Epoch 4/500
154/154 - 201s - 1s/step - loss: 0.0029 - val_loss: 3.4702e-04 - learning_rate: 0.0010
Epoch 5/500
154/154 - 210s - 1s/step - loss: 0.0029 - val_loss: 2.9984e-04 - learning_rate: 0.0010
Epoch 6/500
154/154 - 197s - 1s/step - loss: 0.0029 - val_loss: 3.1795e-04 - learning_rate: 0.0010
Epoch 7/500
154/154 - 205s - 1s/step - loss: 0.0029 - val_loss: 3.8810e-04 - learning_rate: 0.0010
Epoch 8/500
154/154 - 194s - 1s/step - loss: 0.0029 - val_loss: 4.3679e-04 - learning_rate: 0.0010
Epoch 9/500
154/154 - 201s - 1s/step - loss: 0.0029 - val_loss: 3.1394e-04 - learning_rate: 0.0010
Epoch 10/500
154/154 - 200s - 1s/step - loss: 0.0029 - val_loss: 3.6901e-04 - learning_rate: 0.0010
Epoch 11/500


'Entrenando cluster numero: 1'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,learning_rate
  current = self.get_monitor_value(logs)


1/1 - 0s - 162ms/step - loss: 352.3777 - learning_rate: 0.0010
Epoch 4/500
1/1 - 0s - 132ms/step - loss: 318.9904 - learning_rate: 0.0010
Epoch 5/500
1/1 - 0s - 134ms/step - loss: 297.4808 - learning_rate: 0.0010
Epoch 6/500
1/1 - 0s - 118ms/step - loss: 280.5323 - learning_rate: 0.0010
Epoch 7/500
1/1 - 0s - 125ms/step - loss: 266.5002 - learning_rate: 0.0010
Epoch 8/500
1/1 - 0s - 117ms/step - loss: 256.7179 - learning_rate: 0.0010
Epoch 9/500
1/1 - 0s - 118ms/step - loss: 247.3017 - learning_rate: 0.0010
Epoch 10/500
1/1 - 0s - 114ms/step - loss: 239.4632 - learning_rate: 0.0010
Epoch 11/500
1/1 - 0s - 117ms/step - loss: 231.7408 - learning_rate: 0.0010
Epoch 12/500
1/1 - 0s - 122ms/step - loss: 224.4904 - learning_rate: 0.0010
Epoch 13/500
1/1 - 0s - 148ms/step - loss: 218.4551 - learning_rate: 0.0010
Epoch 14/500
1/1 - 0s - 134ms/step - loss: 211.9915 - learning_rate: 0.0010
Epoch 15/500
1/1 - 0s - 133ms/step - loss: 206.3284 - learning_rate: 0.0010
Epoch 16/500
1/1 - 0s - 129ms/s

'Entrenando cluster numero: 14'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
1/1 - 10s - 10s/step - loss: 40.1298 - val_loss: 71.0613 - learning_rate: 0.0010
Epoch 2/500
1/1 - 1s - 562ms/step - loss: 36.9838 - val_loss: 63.6881 - learning_rate: 0.0010
Epoch 3/500
1/1 - 1s - 583ms/step - loss: 31.2499 - val_loss: 58.1144 - learning_rate: 0.0010
Epoch 4/500
1/1 - 1s - 626ms/step - loss: 27.5477 - val_loss: 54.6045 - learning_rate: 0.0010
Epoch 5/500
1/1 - 1s - 650ms/step - loss: 25.8910 - val_loss: 52.1763 - learning_rate: 0.0010
Epoch 6/500
1/1 - 1s - 549ms/step - loss: 25.0314 - val_loss: 50.4244 - learning_rate: 0.0010
Epoch 7/500
1/1 - 1s - 717ms/step - loss: 24.5334 - val_loss: 49.2844 - learning_rate: 0.0010
Epoch 8/500
1/1 - 1s - 624ms/step - loss: 24.0733 - val_loss: 48.5531 - learning_rate: 0.0010
Epoch 9/500
1/1 - 1s - 560ms/step - loss: 23.6529 - val_loss: 48.0684 - learning_rate: 0.0010
Epoch 10/500
1/1 - 1s - 523ms/step - loss: 23.2137 - val_loss: 47.7670 - learning_rate: 0.0010
Epoch 11/500
1/1 - 1s - 641ms/step - loss: 22.6451 - val_los

#### Paso 7: Sumarizar las predicciones y aplicar ratios


In [152]:
scaler_tn = scalers['tn']
predictions = []

for cluster in range(n_clusters):
    if cluster not in models:
        continue

    model = models[cluster]
    cluster_data = grouped_df[grouped_df['cluster'] == cluster].copy()
    
    X_pred_data = []
    keys = []
    for key, data in cluster_data.groupby(['customer_id', 'product_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'quarter', 'month', 'customer_id', 'product_id', 'tn']].values
        max_len = len(series) - 1
        X_pred = np.pad(series[1:], ((max_len - len(series[1:]), 0), (0, 0)), mode='constant').astype(np.float32)
        X_pred_data.append(X_pred)
        keys.append(key)
    
    if len(X_pred_data) == 0:
        continue
    
    max_len_pred = max(len(seq) for seq in X_pred_data)
    X_pred_padded = np.array([np.pad(seq, ((max_len_pred - len(seq), 0), (0, 0)), mode='constant') for seq in X_pred_data]).astype(np.float32)
    X_pred_padded = np.reshape(X_pred_padded, (X_pred_padded.shape[0], X_pred_padded.shape[1], X_pred_padded.shape[2]))
    
    preds = model.predict(X_pred_padded, verbose=0)
    
    for key, pred in zip(keys, preds):
        inverse_pred = scaler_tn.inverse_transform([pred])
        predictions.append([key[0], key[1], inverse_pred[0][0]])
    
    pred_df_temp = pd.DataFrame(predictions, columns=['customer_id', 'product_id', 'prediccion'])
    pred_df_temp.to_csv(f"predicciones_temprales_cluster_pID{cluster}.csv", index=False)

In [153]:
summarized_preds = pred_df_temp.groupby(['product_id'])['prediccion'].sum().reset_index()

final_predictions_df = pd.DataFrame(summarized_preds, columns=['product_id', 'prediccion'])

final_predictions_df.to_csv('predicciones_finales.csv', index=False)

display(final_predictions_df)

,product_id,prediccion
0,20001,1153.132826
1,20002,1157.153841
2,20003,692.881444
3,20004,498.011372
4,20005,535.229001
...,...,...
775,21263,0.982554
776,21265,1.511621
777,21266,1.511621
778,21267,0.579455
